In [5]:
from sklearn.preprocessing import StandardScaler
import os

from TSForecasting.utils.timefeatures import time_features
import pandas as pd
from torch.utils.data import Dataset


class Dataset_Custom(Dataset):
    def __init__(self, root_path, flag, size, features, data_path,
                 target, scale=False, timeenc=0, freq='h',train_only=False):
        
        if size is None:
            self.seq_len = 24 * 4 * 4
            self.label_len = 24 * 4
            self.pred_len = 24 * 4
            
        else:
            self.seq_len = size[0]
            self.label_len = size[1]
            self.pred_len = size[2]
            
        assert flag in ['train', 'test', 'val']
        type_map = {'train': 0, 'val': 1, 'test': 2}
        self.set_type = type_map[flag]
        self.train_only = train_only

        self.features = features
        self.target = target
        self.scale = scale
        self.timeenc = timeenc
        self.freq = freq

        self.root_path = root_path
        self.data_path = data_path
        self.__read_data__()

    def __read_data__(self):
        self.scaler = StandardScaler()
        df_raw = pd.read_csv(os.path.join(self.root_path, self.data_path))
        print(df_raw.columns)
        '''
        df_raw.columns: ['date', ...(other features), target feature]
        '''
        cols = list(df_raw.columns)
        if self.features == 'S':
            cols.remove(self.target)
        cols.remove('date')

        num_train = int(len(df_raw) * (0.7 if not self.train_only else 1))
        num_test = int(len(df_raw) * 0.2)
        num_vali = len(df_raw) - num_train - num_test
        border1s = [0, num_train - self.seq_len, len(df_raw) - num_test - self.seq_len]
        border2s = [num_train, num_train + num_vali, len(df_raw)]
        border1 = border1s[self.set_type]
        border2 = border2s[self.set_type]

        if self.features == 'M' or self.features == 'MS':
            df_raw = df_raw[['date'] + cols]
            cols_data = df_raw.columns[1:]  # Using columns 1 through end column (first col - date)
            df_data = df_raw[cols_data]
            self.num_features = len(cols_data)
        elif self.features == 'S':
            df_raw = df_raw[['date'] + cols + [self.target]]
            df_data = df_raw[[self.target]]
            self.num_features = 1

        if self.scale:
            train_data = df_data[border1s[0]:border2s[0]]
            self.scaler.fit(train_data.values)
            # print(self.scaler.mean_)
            # exit()
            data = self.scaler.transform(df_data.values)
        else:
            data = df_data.values

        df_stamp = df_raw[['date']][border1:border2]
        df_stamp['date'] = pd.to_datetime(df_stamp.date)
        if self.timeenc == 0:
            df_stamp['month'] = df_stamp.date.apply(lambda row: row.month, 1)
            df_stamp['day'] = df_stamp.date.apply(lambda row: row.day, 1)
            df_stamp['weekday'] = df_stamp.date.apply(lambda row: row.weekday(), 1)
            df_stamp['hour'] = df_stamp.date.apply(lambda row: row.hour, 1)
            data_stamp = df_stamp.drop(['date'], axis=1).values
            
        elif self.timeenc == 1:
            data_stamp = time_features(pd.to_datetime(df_stamp['date'].values), freq=self.freq)
            data_stamp = data_stamp.transpose(1, 0)

        self.data_x = data[border1:border2]
        self.data_y = data[border1:border2]
        self.data_stamp = data_stamp

    def __getitem__(self, index):
        s_begin = index
        s_end = s_begin + self.seq_len
        r_begin = s_end - self.label_len
        r_end = r_begin + self.label_len + self.pred_len

        seq_x = self.data_x[s_begin:s_end]
        seq_y = self.data_y[r_begin:r_end]
        seq_x_mark = self.data_stamp[s_begin:s_end]
        seq_y_mark = self.data_stamp[r_begin:r_end]

        return seq_x, seq_y, seq_x_mark, seq_y_mark

    def __len__(self):
        return len(self.data_x) - self.seq_len - self.pred_len + 1

    def inverse_transform(self, data):
        return self.scaler.inverse_transform(data)
    

In [6]:
data_set = Dataset_Custom(root_path='.', data_path='data/processed/train_set_processed.csv', flag='train',
                size=[4, 0, 1], features='MS', target='orders', timeenc=0, freq='d')


Index(['warehouse', 'date', 'holiday', 'shops_closed',
       'winter_school_holidays', 'school_holidays', 'orders', 'day', 'month',
       'year', 'day_of_week', 'week_of_year', 'is_weekend', 'is_holiday',
       'quarter'],
      dtype='object')


/tmp/ipykernel_208453/1656254287.py:80: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  df_stamp['month'] = df_stamp.date.apply(lambda row: row.month, 1)
/tmp/ipykernel_208453/1656254287.py:81: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  df_stamp['day'] = df_stamp.date.apply(lambda row: row.day, 1)
/tmp/ipykernel_208453/1656254287.py:82: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  df_stamp['weekday'] = df_stamp.date.apply(lambda row: row.weekday(), 1)
/tmp/ipykernel_208453/1656254287.py:83: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  D

In [8]:
data_set.data_x

array([[4., 0., 0., ..., 1., 0., 4.],
       [4., 0., 0., ..., 1., 0., 4.],
       [4., 0., 0., ..., 0., 0., 4.],
       ...,
       [3., 0., 0., ..., 0., 0., 4.],
       [3., 0., 0., ..., 0., 0., 4.],
       [3., 0., 0., ..., 0., 0., 4.]])

In [10]:
data_set.data_y

array([[4., 0., 0., ..., 1., 0., 4.],
       [4., 0., 0., ..., 1., 0., 4.],
       [4., 0., 0., ..., 0., 0., 4.],
       ...,
       [3., 0., 0., ..., 0., 0., 4.],
       [3., 0., 0., ..., 0., 0., 4.],
       [3., 0., 0., ..., 0., 0., 4.]])

In [11]:
from torch.utils.data import DataLoader

data_loader = DataLoader(data_set, batch_size=2, shuffle=True, num_workers=4, drop_last=True)

for i in data_loader:
    break

In [13]:
i[0]

tensor([[[4.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
          9.5050e+03, 2.0000e+00, 2.0000e+00, 2.0220e+03, 2.0000e+00,
          5.0000e+00, 0.0000e+00, 0.0000e+00, 1.0000e+00],
         [4.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
          9.5840e+03, 3.0000e+00, 2.0000e+00, 2.0220e+03, 3.0000e+00,
          5.0000e+00, 0.0000e+00, 0.0000e+00, 1.0000e+00],
         [4.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
          1.0411e+04, 4.0000e+00, 2.0000e+00, 2.0220e+03, 4.0000e+00,
          5.0000e+00, 0.0000e+00, 0.0000e+00, 1.0000e+00],
         [4.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
          8.8280e+03, 5.0000e+00, 2.0000e+00, 2.0220e+03, 5.0000e+00,
          5.0000e+00, 1.0000e+00, 0.0000e+00, 1.0000e+00]],

        [[6.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
          4.6610e+03, 2.5000e+01, 4.0000e+00, 2.0220e+03, 0.0000e+00,
          1.7000e+01, 0.0000e+00, 0.0000e+00, 2.0000e+00],
   

In [14]:
i[1]

tensor([[[4.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
          8.9590e+03, 6.0000e+00, 2.0000e+00, 2.0220e+03, 6.0000e+00,
          5.0000e+00, 1.0000e+00, 0.0000e+00, 1.0000e+00]],

        [[6.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
          5.1420e+03, 2.9000e+01, 4.0000e+00, 2.0220e+03, 4.0000e+00,
          1.7000e+01, 0.0000e+00, 0.0000e+00, 2.0000e+00]]],
       dtype=torch.float64)

In [9]:
pd.read_csv('data/processed/train_set_processed.csv')

,warehouse,date,holiday,shops_closed,winter_school_holidays,school_holidays,orders,day,month,year,day_of_week,week_of_year,is_weekend,is_holiday,quarter
0,4,2020-12-05,0,0,0,0,6895.0,5,12,2020,5,49,1,0,4
1,4,2020-12-06,0,0,0,0,6584.0,6,12,2020,6,49,1,0,4
2,4,2020-12-07,0,0,0,0,7030.0,7,12,2020,0,50,0,0,4
3,4,2020-12-08,0,0,0,0,6550.0,8,12,2020,1,50,0,0,4
4,4,2020-12-09,0,0,0,0,6910.0,9,12,2020,2,50,0,0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7335,1,2024-03-10,0,0,0,0,6733.0,10,3,2024,6,10,1,0,1
7336,1,2024-03-11,0,0,0,0,6492.0,11,3,2024,0,11,0,0,1
7337,1,2024-03-12,0,0,0,0,6661.0,12,3,2024,1,11,0,0,1
7338,1,2024-03-13,0,0,0,0,6843.0,13,3,2024,2,11,0,0,1
